In [1]:
from SD_IB_IRP_PPenv import steroid_IRP
from random import choice, randint
from termcolor import colored

rd_seed = 0

### Auxiliary functions for the demo
def print_valid(statement):  
    if statement:  return 'Passed', 'green'
    else: return 'Failed', 'red'

def gen_generic_dic(env):
    ret_dic = {(k,o): 0 for k in env.Suppliers for o in env.Ages[k]}
    return ret_dic

def print_state(env):
    state = 'State: \n'
    
    for k in env.Products:
        state += f'Product {k}: '
        for o in env.Ages[k]:   state += f'age {o}: {env.state[k,o]}; '
        if env.others['back_orders'] == 'back-logs':    state += f'BL: {env.state[k,"B"]}\n'
        else:   state += '\n'

    return state 

# Stochastic-Dynamic Inventory-Routing-Problem with Perishable Products environment
by: Juan Betancourt

## Powelleskian model

### State ($S_t$)
-   $R_t$ **Physical State**:

        state: Current inventory (dict) - Inventory of k \in K of age o \in O_k

-   $I_t$ **Other deterministic info**:

        p: Prices (dict) - Price of k \in K at i \in M
    
        q: Available quantities (dict) - Available quantities of k \in K at i \in M
    
        h: Holding cost (dict) - Holding cost of k \in K
    
        historic_data (dict) - Historic log of information (optional)
    
-   $B_t$ **Belief State**:
    
        sample_paths: Sample paths


### Actions ($X_t$)
The action can be seen as a three level-decision. These are the three layers:

1. Routes to visit selected suppliers

2. Quantity to purchase on each supplier

3. Demand complience plan, dispatch decision

Accordingly, the action will be a list composed as follows:

$$ X = [\text{routes, purchase, demand complience}] $$

        - routes (list): list of list with the nodes visited on a route (including departure and arriving to the depot)

        - purchase (dict): Units to purchase of k \in K at i \in M

        - demand_complience (dict): Units to use of k in K of age o \in O_k


## INITIALIZATION

### Time Horizon: Two time horizon types (horizon_type = 'episodic')

1. 'episodic': Every episode (simulation) has a finite number of steps
    
    Related parameters:
        
        - T: Decision periods (time-steps)
    
        
2. 'continuous': Neverending episodes
    
    Related parameters: 
        
        - gamma: Discount factor

(For internal environment's processes: 1 for episodic, 0 for continouos)

In [2]:
horizon_type = 'episodic'

T = 4

### Look-ahead approximation: Generation of sample paths (look_ahead = ['d']):

1. List of parameters to be forcasted on the look-ahead approximation ['d', 'p', ...]

2. List with '*' to generate forecasts for all parameters

3. False for no sample path generation

Related parameters:

    - S: Number of sample paths
    
    - LA_horizon: Number of look-ahead periods

In [3]:
look_ahead = ['*']

S = 2
LA_horizon = 3

### Historic data: Generation or usage of historic data (historic_data = ['d'])   

1. ['d', 'p', ...]: List with the parameters the historic info will be generated for

2.  ['*']: Historic info generated for all parameters

3. !!! NOT DEVELOPED path: File path to be processed by upload_historic_data() 

4.  False: No historic data will be used

Related parameter:
    
    - hist_window: Initial log size (time periods)

In [4]:
historic_data = ['*']

hist_window = 10

### Back-orders: Catch unsatisfied demand (back_orders = False):

1. 'back-orders': Demand can be not fully satisfied. Non-complied orders will be automatically fullfilled with an extra-cost

2. 'back-logs': Demand can be not fully satisfied. Non-complied orders will be registered and kept track of

3. False: All demand must be fullfilled

Related parameter:

    - back_o_cost = 20
    - back_l_cost = 20

In [5]:
back_orders = 'back-orders'

back_l_cost = 20

### Other customizable parameters

    -   M = 10: Number of suppliers

    -   K = 10: Number of Products

    -   F = 2:  Number of vehicles on the fleete

    -   T = 6:  Number of decision periods

    -   wh_cap = 1e9: Warehouse capacity

    -   min/max_sprice: Max and min selling prices (per m and k)

    -   min/max_hprice: Max and min holding cost (per k)

    -   penalization_cost: Penalization costs for RL (invalid actions, etc.)

    -   S = 4:  Number of sample paths 

    -   LA_horizon = 5: Number of look-ahead periods

    -   lambda1 = 0.5: Controls demand, assures feasibility

In [6]:
env_config = {  'M': 3, 
                'K': 3, 
                'T': T, 
                'F': 2, 
                
                'min_sprice': 1, 
                'max_sprice': 500, 
                'min_hprice': 1, 
                'max_hprice': 500, 
                'back_l_cost': 20,
                
                'S': S, 
                'LA_horizon': LA_horizon, 
                'lambda1': 0.5
            }
            

# Creating an environment

The environment receives all the previous parameters plus a random seed as the parameters and a customizable parameter env_config with the specified characteristics
    
    -   rd_seed: Seed for random number generation

    -   env_config: Receives a dictionary with custom environment parameters

In [7]:
env = steroid_IRP(  horizon_type = horizon_type, 
                    look_ahead = look_ahead, 
                    historic_data = historic_data, 
                    back_orders = back_orders,
                    rd_seed = rd_seed, 
                    env_config = env_config)
repr(env)

'Stochastic-Dynamic Inventory-Routing-Problem with Perishable Products instance. V = 3; K = 3; F = 2'

# Reseting the environment

Once the environment is created, or everytime it will be run again from the start, it must be reset. For this, the class has the step method which receives a boolean under the parameter:

    -   return_state: Indicates if reset() must return the initial state

In [8]:
return_state = False
env.reset(return_state = return_state)

# Retrieving information from the environment

Once the environmnet has been reset, all the initial values can be accesed

    - Inventory[k,o]

In [9]:
print(f'{print_state(env)} \n')

product = choice(env.Products); age = randint(1, env.O_k[product])
#print(env.O_k[product])
print(f'The inventory of product {product} and age {age} is {env.state[product,age]}')

State: 
Product 0: age 1: 0; age 2: 0; age 3: 0; age 4: 0; 
Product 1: age 1: 0; age 2: 0; age 3: 0; age 4: 0; 
Product 2: age 1: 0; 
 

The inventory of product 0 and age 2 is 0


    -   Available quantiites [i,k]
    
    -   Prices [i,k]
    
    -   Holding cost [k]
    
    -   Demand [k]

In [10]:
#print(env.q, '\n')
supplier = choice(env.Suppliers); product = choice(env.Products)
print(f'Supplier {supplier} offers {env.q[supplier, product]} of product {product}')

#print(env.p, '\n')
print(f'Supplier {supplier} offers product {product} at ${env.p[supplier, product]}')

#print(env.h, '\n')
product = choice(env.Products)
print(f'Holding cost for product {product} is ${env.h[product]}')

#print(env.h, '\n')
product = choice(env.Products)
print(f'Demand for product {product} is {env.d[product]}')

Supplier 1 offers 11 of product 1
Supplier 1 offers product 1 at $264
Holding cost for product 0 is $295
Demand for product 0 is 10.0


Historic data

    -   Available quantiites
    
    -   Prices
    
    -   Holding cost 
    
    -   Demand

In [11]:
supplier = choice(env.Suppliers); product = choice(env.Products)

historic_quantities = env.historic_data['q'][supplier, product]
print(f'The historic a.q. for supplier {supplier} on product {product} are {historic_quantities} \n')

historic_demand = env.historic_data['d'][product]
print(f'The historic demand of produdct {product} is {historic_demand}')

The historic a.q. for supplier 1 on product 1 are [0, 0, 8, 8, 6, 14, 0, 13, 0, 13, 0, 0, 6, 2, 8, 2, 12, 0, 8, 0, 0, 0, 7, 1, 5, 6, 12, 0, 11, 15, 3, 0, 0, 15, 3, 11, 0, 10, 0, 0] 

The historic demand of produdct 1 is [0.0, 0.0, 18.0, 18.0, 6.0, 14.0, 0.0, 20.5, 0.0, 19.0, 1.0, 0.0, 18.0, 2.0, 13.0, 9.0, 19.0, 0.0, 15.0, 4.0, 0.0, 0.0, 8.0, 1.0, 10.5, 10.0, 14.5, 4.0, 11.0, 20.5, 3.5, 0.0, 0.0, 15.0, 14.5, 13.0, 0.0, 10.0, 10.0, 3.0]


Sample paths

    -   Available quantiites
    
    -   Prices
    
    -   Holding cost 
    
    -   Demand

In [12]:
sample = choice(env.Samples)
proy_day = randint(1, env.LA_horizon - 1)
supplier = choice(env.Suppliers); product = choice(env.Products)

proy_quant = env.sample_paths[('q',sample)][(supplier, product, proy_day)]
print(f'On sample path {sample} the forcasted available quantity of product {product} on supplier {supplier} for day {proy_day} is {proy_quant}')

proy_demand = env.sample_paths[('d',sample)][product, proy_day]
print(f'On sample path {sample} the forcasted demand of product {product} for day {proy_day} is {proy_demand}')

On sample path 1 the forcasted available quantity of product 0 on supplier 1 for day 1 is 0
On sample path 1 the forcasted demand of product 0 for day 1 is 0.0


# Safe check

    - First day of forecast is the realized random value

In [13]:
sample = choice(env.Samples)
proy_day = 0
supplier = choice(env.Suppliers); product = choice(env.Products)

test_q, col_q = print_valid(env.sample_paths["q",sample][supplier,product,proy_day] == env.q[supplier,product])
print('Test q:', colored(test_q, col_q))
test_p, col_p = print_valid(env.sample_paths["p",sample][supplier,product,proy_day] == env.p[supplier,product])
print('Test p:', colored(test_p, col_p))
test_h, col_h = print_valid(env.sample_paths["h",sample][product,proy_day] == env.h[product])
print('Test h:', colored(test_h, col_h))
test_d, col_d = print_valid(env.sample_paths["d",sample][product,proy_day] == env.d[product])
print('Test h:', colored(test_d, col_d))

Test q: Passed
Test p: Passed
Test h: Passed
Test h: Passed


# Step

Information from the intial state is retrieved. 

In [14]:
print(f'######################################## Time step {env.t} ########################################')
print('   (prod,edad)')
print(f's_{env.t}: {env.state}')
print(f'd_{env.t}: {env.d}')
print(f'q_{env.t}: {env.q}')

# x = env.historic_data['q'][1,0]
# print(f'Historic of a.q is: {x}')
# print(f'A.q is {env.q[1,0]}') 


######################################## Time step 0 ########################################
   (prod,edad)
s_0: {(0, 1): 0, (0, 2): 0, (0, 3): 0, (0, 4): 0, (1, 1): 0, (1, 2): 0, (1, 3): 0, (1, 4): 0, (2, 1): 0}
d_0: {0: 10.0, 1: 16.0, 2: 0.0}
q_0: {(1, 0): 8, (1, 1): 11, (1, 2): 0, (2, 0): 4, (2, 1): 10, (2, 2): 0}


An arbitrary feasible action is generated and its cost is computed 

In [15]:
# Visiting all the suppliers
routes = [[0,1,2,0]]

# Purchase exact quantity for 
purchase = {(1,0): 7, (2,0): 4,     # product 0: 11 units
            (1,1): 8, (2,1): 10,    # product 1: 18 units
            (1,2): 0, (2,2): 0}     # product 2:  0 units

# Demand complience
demand_complience = {(0,0): 10, (0,1): 0, (0,2): 0, (0,3): 0, (0,4): 0,
                     (1,0): 16, (1,1): 0, (1,2): 0, (1,3): 0, (1,4): 0,
                     (2,0): 0,  (2,1): 0}

back_logs_complience = {(0,0): 0, (0,1): 0, (0,2): 0, (0,3): 0, (0,4): 0,
                        (1,0): 0, (1,1): 0, (1,2): 0, (1,3): 0, (1,4): 0,
                        (2,0): 0,  (2,1): 0}

X = [routes, purchase, demand_complience, back_logs_complience]

transport_cost = env.c[routes[0][0], routes[0][1]] + env.c[routes[0][1], routes[0][2]] + env.c[routes[0][2], routes[0][3]]
purchase_cost = 0
for i in env.Suppliers:
    for k in env.Products:
        purchase_cost += purchase[i,k] * env.p[i,k]
holding_cost = env.h[1] * 2 + env.h[0] * 1
total_cost = transport_cost + purchase_cost + holding_cost 
print(f'The total cost of the action is: {total_cost}')

The total cost of the action is: 8945


With a **valid** action, the step method can be called. This method returns:
    
    -   state: New state
    -   reward: The total cost of the action (transport, purchase and holding)
    -   done: Indicates if the episode has finished
    -   _: Extra information 

In [16]:
state, reward, done, _ = env.step(action = X, validate_action = True)

print(f'The computated cost of the action is {reward}')
print(f'Episode finished: {done} \n')

print(f'######################################## Time step {env.t} ########################################')
print(f's_{env.t}: {env.state}')
print(f'd_{env.t}: {env.d}')
print(f'q_{env.t}: {env.q}')

# x = env.historic_data['q'][1,0]
# print(f'Historic of a.q is: {x}')
# print(f'A.q is {env.q[1,0]}')

Back-orders: 0.0
The computated cost of the action is 8945.0
Episode finished: False 

######################################## Time step 1 ########################################
s_1: {(0, 1): 1, (0, 2): 0, (0, 3): 0, (0, 4): 0, (1, 1): 2, (1, 2): 0, (1, 3): 0, (1, 4): 0, (2, 1): 0}
d_1: {0: 13.0, 1: 20.5, 2: 4.0}
q_1: {(1, 0): 8, (1, 1): 11, (1, 2): 0, (2, 0): 9, (2, 1): 15, (2, 2): 4}


In [17]:
# Visiting all the suppliers
routes = [[0,1,2,0]]

# Purchase exact quantity for 
purchase = {(1,0): 8,  (2,0): 9,    # product 0: 17 units
            (1,1): 11, (2,1): 15,   # product 1: 26 units
            (1,2): 0,  (2,2): 4}    # product 2: 4 units

# Demand complience
demand_complience = {(0,0): 13, (0,1): 0, (0,2): 0, (0,3): 0, (0,4): 0,
                     (1,0): 19.5, (1,1): 1, (1,2): 0, (1,3): 0,  (1,4): 0,
                     (2,0): 4, (2,1): 0}

X = [routes, purchase, demand_complience]

transport_cost = env.c[routes[0][0], routes[0][1]] + env.c[routes[0][1], routes[0][2]] + env.c[routes[0][2], routes[0][3]]
purchase_cost = 0
for i in env.Suppliers:
    for k in env.Products:
        purchase_cost += purchase[i,k] * env.p[i,k]
holding_cost = 5 * env.h[0] + 7.5 * env.h[1]
total_cost = transport_cost + purchase_cost + holding_cost 
print(f'The total cost of the action is: {total_cost}')

The total cost of the action is: 21861.5


In [18]:
state, reward, done, _  = env.step(action = X, validate_action = True)
print(f'The computated cost of the action is {reward}')
print(f'Episode finished: {done} \n')

print(f'######################################## Time step {env.t} ########################################')
print(f's_{env.t}: {env.state}')
print(f'd_{env.t}: {env.d}')
print(f'q_{env.t}: {env.q}')

# x = env.historic_data['q'][1,0]
# print(f'Historic of a.q is: {x}')
# print(f'A.q is {env.q[1,0]}')

Back-orders: 0.0
The computated cost of the action is 21861.5
Episode finished: False 

######################################## Time step 2 ########################################
s_2: {(0, 1): 4, (0, 2): 1, (0, 3): 0, (0, 4): 0, (1, 1): 6.5, (1, 2): 1, (1, 3): 0, (1, 4): 0, (2, 1): 0}
d_2: {0: 0.0, 1: 0.0, 2: 1.0}
q_2: {(1, 0): 0, (1, 1): 0, (1, 2): 0, (2, 0): 0, (2, 1): 0, (2, 2): 1}


Let's try some invalid actions on the environment
